In [10]:
using DelimitedFiles

function read_points(filename)
    raw = read(filename, String)

    # Remove the enclosing double braces {{ ... }}
    inner = raw[3:end-2]          # drop first two chars "{{" and last "}}"

    # Split into individual vector strings
    vecs = split(inner, "},{")

    # Parse each vector
    points = [parse.(Float64, split(replace(v, ['{','}'] => ""), ",")) for v in vecs]

    # Convert to a matrix (Nx3)
    return reduce(vcat, (p' for p in points))
end

pts = read_points("A_sensors_data.txt")


using LinearAlgebra
function distance_table(pts)
    n = size(pts,1)
    D = zeros(Float64, n, n)
    for i in 1:(n-1)
        for j in (i+1):n
            D[i,j] = D[j, i] = norm(pts[i, :]-pts[j, :])
        end
    end
    return D
end


function VR(r, D)
    edges = Tuple{Int,Int}[]
    n=size(D,1)
    for i in 1:(n-1)
        for j in (i+1):n
            if D[i,j] ≤ r 
                push!(edges, (i,j))
            end
        end
    end
    return edges
end

VR (generic function with 1 method)

In [11]:
pts=[1 0 0; 0 0 1; 0 1 0]

3×3 Matrix{Int64}:
 1  0  0
 0  0  1
 0  1  0

In [12]:
#my hw1 code for connected components
function edgedictionary(V, E)
    n = V[end]
    # set up the dictionary (empty sets of k-tuples in each dimension)
    edge_dict = Dict(k => Set{Int}() for k in 1:n)
    for (v1, v2) in E
        push!(edge_dict[v1], v2)
        push!(edge_dict[v2], v1)
    end
    return edge_dict
end

function component_of(edge_dict, vertex, visited=Set{Int}())
    push!(visited, vertex)
    component = [vertex]

    for neighbor in edge_dict[vertex]
        if !(neighbor in visited)
            append!(component, component_of(edge_dict, neighbor, visited))
        end
    end
    return sort(component)
end

function findcomponents(V, E)
    edge_dict = edgedictionary(V, E)

    visited = Set{Int}()
    components = []

    for vertex in V
        if !(vertex in visited)
            push!(components, component_of(edge_dict, vertex, visited))
        end
    end
    return components
end

findcomponents (generic function with 1 method)

In [13]:
length(findcomponents([1,2,3],[(1,2),(1,3)]))

1

In [14]:
function is_connected(V,E; exclude=Set{Int}())
    if isempty(exclude)
        comp = findcomponents(V,E)
    else
        E=[e for e in E if !(e[1] in exclude || e[2] in exclude)]
        comp = findcomponents(V,E)
        comp =[c for c in comp if all(x -> !(x in exclude), c)]
    end
    return (length(comp) == 1)
end

function find_r(pts; steps = 1000)
    D = distance_table(pts) 
    V = collect(1:(size(pts,1)))
    lo = 0.0
    hi = maximum(D)
    for _ in 1:steps
        r = (lo + hi) / 2
        edges = VR(r,D)
        if is_connected(V, edges)
            hi = r
        else
            lo = r
        end
    end
    return (lo + hi) / 2
end

find_r (generic function with 1 method)

In [15]:
r=find_r(pts)


1.414213562373095

r=
0.4965033379263023


In [16]:
D=distance_table(pts)
V = collect(1:(size(pts,1)))
E=VR(r,D)
is_connected(V,E,exclude=Set([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]))

false

This must be run in julia and not jupyter as display doesnt work in notebook

In [ ]:
using WGLMakie, ColorTypes

function plot_sphere_with_points_web(pts, edges)
    fig = Figure(size=(800, 800))
    ax = Axis3(fig[1,1], aspect=:data)

    # Sphere mesh
    θ = range(0, 2π, length=80)
    φ = range(0, π, length=40)
    x = [sin(phi)*cos(th) for phi in φ, th in θ]
    y = [sin(phi)*sin(th) for phi in φ, th in θ]
    z = [cos(phi) for phi in φ, th in θ]

    # Make sphere transparent
    surface!(ax, x, y, z, color=RGBA(0.5, 0.7, 1.0, 0.2), transparency=true)

    # Points
    scatter!(ax, pts[:,1], pts[:,2], pts[:,3], color=RGBA(1,0,0,1), markersize=15)

    # Edges
    for (i,j) in edges
        lines!(ax, [pts[i,1], pts[j,1]], [pts[i,2], pts[j,2]], [pts[i,3], pts[j,3]], color=:black, linewidth=2)
    end

    fig
end

fig = plot_sphere_with_points_web(pts, edges)
fig  

┌ Warning: Port in use, using different port. New port: 9385
└ @ Bonito.HTTPServer C:\Users\Uporabnik\.julia\packages\Bonito\18mTs\src\HTTPServer\implementation.jl:346


MethodError: MethodError: no method matching plot_sphere_with_points(::Matrix{Int64}, ::Vector{Tuple{Int64, Int64}}; azimuth::Float64, elevation::Float64)
This method does not support all of the given keyword arguments (and may not support any).

Closest candidates are:
  plot_sphere_with_points(::Any, ::Any) got unsupported keyword arguments "azimuth", "elevation"
   @ Main c:\Users\Uporabnik\Documents\GitHub\Senzorji-topology\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X11sZmlsZQ==.jl:14
